# 推薦模型

## 前言(通常-->廢話)
早餐去買三明治的時候，老闆娘總會好心的問說，要不要來杯紅茶。通常看在老闆娘的~~顏值~~誠意上，都會不爭氣的買了...推薦是針對原本客戶沒有預期到的結果，額外的跨售商品，或提示給用戶可能感興趣（喜歡）的東西（例如:買早餐時原本不會口渴，受到老闆娘的~~顏值~~影響才加購飲料的。）

推薦方法火熱了這麼多年是因為在商業上有很實際的目的：

1. 促進交易
2. 加速決策

簡單來說對商家來說，就是賺更多`$$`。對消費者來說可以從雜亂無章，一堆無用訊息裡面過濾有興趣的商品。在資訊過載的世界裡面，消費者無法一下子就能找到感興趣的東西。谷歌搜尋技法，在目的不明確的狀況下，沒辦法發揮功能。

在網路時代，商家通常能夠快速的蒐集到用戶的購買/瀏覽/蒐藏紀錄，讓蒐集到的資訊，透過模型建立有機會，找到相似（相關性高）的商品來推薦（推播）。建構在模型上，消費者體驗的整體流程可以稱之為系統，系統面結合的範圍較大不在本文討論範圍（系統架構面本人也不懂，只懂簡單的幾種模型）。這一系列的文章，希望能紀錄自己對幾種熱門模型的理解。

一般來說（就本人所知）有以下的算法框架
_______

## 算法框架

* 協同過濾法(Collaborative Filtering)
    - 相似度（最鄰近法-KNN)
        - ubcf, ibcf
    - 矩陣分解(Matrix Factorization)
        - explict (implicit) ALS
        - SVD, SVD++, SVDFeatures ...
        - Learning to rank (BSR,WARP...)
    - 深度學習
        - DNN
* 以內容最基礎(Content-based)
_____________________________

## 最鄰近法(相似度)
協同過濾法簡單說，就是透過群體的交易行為來推算，小王購買A,B,C的可能機會。透過群體關係，來推論誰和你的購買歷程相近...最大的特色是，不需要對商品/領域有專業知識即能有相當的準確度。其中最鄰近法(KNN)以考慮最相鄰的N人(物品)來計算，不算是一種[模型]，只是一種群眾給出的統計結果。沒有模型常見的訓練過程

1. 定義cost function,
2. 求得最佳參數
3. 評估模型結果

有以交易紀錄形成的資料如下，

| |趙 |錢 |孫 | 李|
|---|---|---|---|---|
|牙刷|null|V|null|V|
|牙膏|null|V|null|null|
|腳踏車|null|null|V|V|
|LED燈|V|null|null|V|
|澡盆|V|V|null|V|

以商品的角度來看，牙刷的交易紀錄`(0,1,0,1)`形成空間一組向量，可以計算與牙膏的相似度`(0,1,0,0)`。能建構所有商品的相似度表格

$$
cos(\theta) = \frac{r_{牙膏} \cdot r_{牙刷}'}{\lVert{r_{牙膏}}\lVert \lVert r_{牙刷}' \lVert}
$$

In [10]:
import numpy as np

def cos_similarity(rating,kind='ubcf',eps=1e-9):
    if kind =='ubcf':
        sim = np.dot(rating,rating.T) + eps
    if kind =='ibcf':
        sim = np.dot(rating.T,rating) + eps
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return sim/norms/norms.T

rating_t = np.array([
    [0,1,0,1],
    [0,1,0,0],
    [0,0,1,1],
    [1,0,0,1],
    [1,1,0,1]
  ])
rating = rating_t.T
sim_i = similarity(rating,kind='ibcf')
sim_i.astype(np.float16)

array([[ 1.        ,  0.70703125,  0.5       ,  0.5       ,  0.81640625],
       [ 0.70703125,  1.        ,  0.        ,  0.        ,  0.57714844],
       [ 0.5       ,  0.        ,  1.        ,  0.5       ,  0.40820312],
       [ 0.5       ,  0.        ,  0.5       ,  1.        ,  0.81640625],
       [ 0.81640625,  0.57714844,  0.40820312,  0.81640625,  1.        ]], dtype=float16)

有相似度矩陣後，就能針對每個用戶還沒買的商品作預測。

$$
\hat{r_{ui}} = \sum_{u'}Sim(u,u') \cdot r_{u'i}
$$
or 
$$
\hat{r_{ui}} = \sum_{i'} r_{ui'} \cdot Sim(i,i')
$$

In [17]:
score_ibcf = rating.dot(sim_i).astype(np.float16)
score_ibcf

array([[ 1.31640625,  0.57714844,  0.90820312,  1.81640625,  1.81640625],
       [ 2.5234375 ,  2.28515625,  0.90820312,  1.31640625,  2.39453125],
       [ 0.5       ,  0.        ,  1.        ,  0.5       ,  0.40820312],
       [ 2.81640625,  1.28417969,  2.40820312,  2.81640625,  3.04101562]], dtype=float16)

以物品為基礎

    - 牙膏澡盆相似度最高(0.816),牙膏牙刷相似度次高(0.7)
    - 推薦孫：牙刷(0.5), LED燈(0.5) 已買腳踏車
____________________    

## 推薦實例Sketchfab Demo

我學習推薦系算法，除了阿撒不魯的網站論文看了一大堆，後來咀嚼思考後，覺得大概有87%的想法，源碼精神是來自於[DataPique](http://blog.ethanrosenthal.com/)大大的網站（其他的可謂~~垃圾?~~超越小弟~~腦弱~~的理解)，這個大大除了把最重要的幾種模型詳細推導一遍，更有趣的是作者(ethan)更自己爬了3DCAD的資料，蒐集第一手的客戶資料情況。真正公司會在網站後蒐集到的資料，通常都是客戶的隱式行為...(所以用這組資料來練習是最好不過的!)

隱性資料(implicit data)是大部分人會在瀏覽網站的狀況，
1. 好棒棒的點讚，
2. 覺得很爛/很腦殘~~智缺~~/沒興趣，通常是不會有反應的（空值）。

對真實世界來講，通常沒有好棒棒的評分資料，給你好棒棒的評分（像是啥鬼MovieLens影評資料...)。老實說我在聽mixerbox,spotify,netflix不好聽(看)就下一首(片)或關掉了。只有聽到好棒棒的影視/音樂，才會點讚讚。

之後會模仿DataPique的作法，在這個[資料](https://github.com/ihongChen/PlayRecommendSystem/tree/master/rec-a-sketch)上利用KNN來實踐一次。人工智慧看看推薦結果如何...


### 資料馬殺雞
---最囉唆的就是資料清理了---

* 資料通常很稀疏，要用`scipy.sparse`下面的稀疏矩陣來建構資料。
    - [scipy lecture note](http://www.scipy-lectures.org/advanced/scipy_sparse/index.html)有教學說這咪一大堆的稀疏矩陣差異性(~~累死寶寶~~)。
    

In [1]:
import numpy as np 
import pandas as pd
import csv
import sys
import importlib

In [2]:
sys.path.append('../')

In [3]:
from KNNmodel import *
from rec_helper import *

In [4]:
df = pd.read_csv('../rec-a-sketch/model_likes_anon.psv',
                 sep='|',quotechar='\\',quoting=csv.QUOTE_MINIMAL)
df.head()

,modelname,mid,uid
0,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,7ac1b40648fff523d7220a5d07b04d9b
1,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,2b4ad286afe3369d39f1bb7aa2528bc7
2,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,1bf0993ebab175a896ac8003bed91b4b
3,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,6484211de8b9a023a7d9ab1641d22e7c
4,3D fanart Noel From Sora no Method,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,1109ee298494fbd192e27878432c718a


刪除部份重複資料

In [5]:
print(df.count())
df.drop_duplicates(inplace=True)
print(df.count())

modelname    632832
mid          632832
uid          632832
dtype: int64
modelname    632677
mid          632677
uid          632677
dtype: int64


移除喜歡次數過少(`<5`)的用戶

In [6]:
df = threshold_interaction(df,rowname='uid',colname='mid',row_min=5)

Starting interactions info
Number of rows: 62583
Number of cols: 28806
Sparsity: 0.04%
Ending interactions info
Number of rows: 13529
Number of columns: 28707
Sparsity: 0.14%


In [7]:
inter,uid_to_idx,idx_to_uid,mid_to_idx,idx_to_mid=df_to_spmatrix(df,'uid','mid')

In [8]:
## train,test split
train,test, user_idxs = train_test_split(inter,split_count=1,fraction=0.2)

In [10]:
mat = train.T
row_sums = mat.getnnz(axis=1).astype('int16')
ab = mat.dot(mat.T)
ab = ab.astype('float16')
aa = np.repeat(row_sums,ab.getnnz(axis=1))
bb = row_sums[ab.indices]
sim = ab.copy()
sim.data /= (aa+bb-ab.data)

In [15]:
# print(.dtype)
sim.data = sim.data.astype('float32')

In [16]:
sim.setdiag(0)

In [9]:
model_i = KNNmodel(train,kind='ibcf')
model_i.jaccard_sim()

/home/ihong/anaconda3/lib/python3.5/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


similarity (jaccard) matrix built (ibcf), 
sparsity of similarity: 29.14 %


### 建立KNN模型

In [ ]:
model_i = KNNmodel(train,kind='ibcf')
model_i.jaccard_sim()

In [45]:
#ibcf
model_i = KNNmodel(train,kind='ibcf')
model_i.jaccard_sim()
model_i.fit(topK=100,remove=True)
# ## ubcf
model_u = KNNmodel(train,kind='ubcf')
model_u.jaccard_sim()
model_u.fit(topK=100,remove=True)
# ## popular
model_p = KNNmodel(train,kind='popular')
model_p.fit(topK=100,remove=True)

/home/ihong/anaconda3/lib/python3.5/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


similarity (jaccard) matrix built (ibcf), 
sparsity of similarity: 29.14 %


100%|██████████| 28707/28707 [11:04<00:00, 43.23items/s]


ibcf rating matrix built...

handling nan data...


100%|██████████| 15939601/15939601 [00:14<00:00, 1131319.49it/s]


...

numbers of rows : 13529
numbers of cols: 28707
sparsity of rating: 8.82 %
save into *.rating attribute...


100%|██████████| 13529/13529 [00:02<00:00, 6583.04it/s]


popular rating matrix built...

handling nan data...


100%|██████████| 1352900/1352900 [00:01<00:00, 1041182.52it/s]


numbers of rows : 13529
numbers of cols: 115
sparsity of rating: 86.96 %
save into *.rating attribute...


In [66]:
topN = 10 
topNarray = np.zeros((uids.shape[0],topN))
for _idx,_id in enumerate(uids):
    topNarray[_idx,:] = np.argsort(model_i.rating[_id,:].A,kind='heapsort')[:,:-topN-1:-1]


In [69]:
pred_all = topNarray
test_coo = test.tocoo()
score = 0
nonzero_rowsets = set(test_coo.row)
for row,col,v in zip(test_coo.row,test_coo.col,test_coo.data):
    if col in pred_all[row,]:
        score += 1
temp_recall = score/len(nonzero_rowsets)
print("\n-------------")
print("model: {},\ntopN: {}".format('ibcf', topN))
print("recall:{:.2f} %".format(score/len(test_coo.data) * 100))



-------------
model: ibcf,
topN: 10
recall:5.88 %


In [63]:
uids = np.arange(0,train.shape[0])

predall_u = model_u.predict(uids,topN=10)
model_u.evaluate(predall_u,test,method='recall') # 15.49 %

# predall_i = model_i.predict(uids,topN=10)
# model_i.evaluate(predall_i,test,method='precision') #recall:5.88

predall_p = model_p.predict(uids,topN=10)
model_p.evaluate(predall_p,test,method='recall') # 2.62 %


-------------
model: ubcf,
topN: 10
recall:15.49 %

-------------
model: popular,
topN: 10
recall:2.62 %


In [65]:
model_u.evaluate(predall_u,test,method='precision')
model_p.evaluate(predall_p,test,method='precision')


-------------
model: ubcf,
topN: 10
precision: 1.55 %

-------------
model: popular,
topN: 10
precision: 0.26 %
